# Charging stations for electric vehicles

## Libraries and settings

In [10]:
# Libraries
import os
import re
import json
import folium
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen

# Background color of graphics
# plt.style.use('dark_background')

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

c:\Users\drito\Documents\SP\scientific_programming\Week_06\challenge


### Request charging station data

In [11]:
# Url for server request
url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/oicp/ch.bfe.ladestellen-elektromobilitaet.json"
  
# Request data from server
response = urlopen(url)
  
# Load data
data_orig = json.loads(response.read())

# Save the data as pandas data frame
data = pd.DataFrame(data_orig["EVSEData"]) # EVSEData is a list of dictionaries of single charging stations
data

,EVSEDataRecord,OperatorID,OperatorName
0,"[{'AccessibilityLocation': 'OnStreet', 'Addres...",CH*SWISSCHARGE,Swisscharge
1,[{'Accessibility': 'Paying publicly accessible...,CH*CCC,Move
2,"[{'Accessibility': 'Free publicly accessible',...",CH*ECU,eCarUp
3,"[{'EvseID': 'CHFASE4150401', 'Accessibility': ...",CH*FASTNED,Fastned
4,"[{'deltaType': 'insert', 'lastUpdate': '2024-0...",CH*REP,PLUG N ROLL
5,"[{'Accessibility': 'Free publicly accessible',...",CHEVP,evpass
6,"[{'Accessibility': 'Restricted access', 'Acces...",CH*AIL,AIL
7,[{'Accessibility': 'Paying publicly accessible...,CH*ENMOBILECHARGE,ENMOBILECHARGE
8,[{'Accessibility': 'Paying publicly accessible...,CH*EVAEMOBILITAET,EVAEMOBILITAET
9,[{'Accessibility': 'Paying publicly accessible...,CH*EWACHARGE,EWACHARGE


### Deriving data from the operator 'evpass'

In [21]:
# Getting the index of evpass
idx = data[data["OperatorName"].isin(['evpass'])].index[0]
print(idx)

# Create data frame from column 'EVSEDataRecord' in the subset
df = pd.DataFrame(data.loc[idx, 'EVSEDataRecord'])
df.head() # Show the first 5 rows as default, unless you write a number in the ()

5


,Accessibility,Address,DynamicInfoAvailable,EvseID,GeoCoordinates,HotlinePhoneNumber,IsHubjectCompatible,IsOpen24Hours,AdditionalInfo,AuthenticationModes,...,EnergySource,EnvironmentalImpact,LocationImage,SuboperatorName,MaxCapacity,ChargingPoolID,DynamicPowerLevel,HardwareManufacturer,HubOperatorID,AccessibilityLocation
0,Free publicly accessible,"{'City': 'Bussigny', 'Country': 'CHE', 'Street...",true,CHEVPE2,{'Google': '46.547745 6.552618'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
1,Free publicly accessible,"{'City': 'Bussigny', 'Country': 'CHE', 'Street...",true,CHEVPE72,{'Google': '46.547745 6.552618'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
2,Free publicly accessible,"{'City': 'Crans-Montana', 'Country': 'CHE', 'S...",true,CHEVPE92,{'Google': '46.308936 7.471237'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
3,Free publicly accessible,"{'City': 'Crans-Montana', 'Country': 'CHE', 'S...",true,CHEVPE96,{'Google': '46.308936 7.471237'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
4,Free publicly accessible,"{'City': 'Vandoeuvres', 'Country': 'CHE', 'Str...",true,CHEVPE100,{'Google': '46.222434 6.203629'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
5,Free publicly accessible,"{'City': 'Vandoeuvres', 'Country': 'CHE', 'Str...",true,CHEVPE105,{'Google': '46.222434 6.203629'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
6,Free publicly accessible,"{'City': 'Neuchâtel', 'Country': 'CHE', 'Stree...",true,CHEVPE118,{'Google': '46.986632 6.915839'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
7,Free publicly accessible,"{'City': 'Neuchâtel', 'Country': 'CHE', 'Stree...",true,CHEVPE120,{'Google': '46.986632 6.915839'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
8,Free publicly accessible,"{'City': 'Sion', 'Country': 'CHE', 'Street': '...",true,CHEVPE135,{'Google': '46.233099 7.357062'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None
9,Free publicly accessible,"{'City': 'Sion', 'Country': 'CHE', 'Street': '...",true,CHEVPE136,{'Google': '46.233099 7.357062'},+41582219660,True,True,None,"[REMOTE, NFC RFID Classic, NFC RFID DESFire]",...,None,None,None,None,None,None,None,None,None,None


### Take a closer look at the coordinates

In [13]:
# Look at the geo-coordinates (latitude and longitude)
coordinates = df.GeoCoordinates
print(type(coordinates))
print(coordinates)

# Look at the type of a single item in the data frame
print(type(coordinates[0]))

<class 'pandas.core.series.Series'>
0       {'Google': '46.547745 6.552618'}
1       {'Google': '46.547745 6.552618'}
2       {'Google': '46.308936 7.471237'}
3       {'Google': '46.308936 7.471237'}
4       {'Google': '46.222434 6.203629'}
                      ...               
3223      {'Google': '46.38418 6.24537'}
3224    {'Google': '46.385209 6.921363'}
3225    {'Google': '46.035701 8.919653'}
3226    {'Google': '46.215171 6.104451'}
3227    {'Google': '46.215171 6.104451'}
Name: GeoCoordinates, Length: 3228, dtype: object
<class 'dict'>


### Extracting the latitude and longitude

In [14]:
# Getting single values from as string
coord_str = "".join(coordinates[0].values()) # join the values of the dictionary
coord_str #why is it a string? It is a dictionary with key-value

# Splitting string by white space
re.findall(r'\S+', coord_str) # \S+ means one or more non-whitespace characters

['46.547745', '6.552618']

### Using a for loop to write the latitude and longitude to a separate data frame

In [15]:
# Now loop though the coordinates object
out_list = []

for i in coordinates:
    # Getting single values from as string
    coord_str = "".join(i.values())
    # Splitting string by white space
    coords = re.findall(r'\S+', coord_str)
    out_list.append(coords)

# Create data frame from the list
df = pd.DataFrame(out_list, columns =['lat', 'lon'])
print(df.dtypes)

# Here we still have strings which need to be converted to integers

# First: exclude white space using pythons lambda function
df['lat'] = df['lat'].apply(lambda x: x.strip())
df['lon'] = df['lon'].apply(lambda x: x.strip())

# Second: cange to integer values
df['lat'] = df['lat'].astype(float)
df['lon'] =  df['lon'].astype(float)
df

lat    object
lon    object
dtype: object


,lat,lon
0,46.547745,6.552618
1,46.547745,6.552618
2,46.308936,7.471237
3,46.308936,7.471237
4,46.222434,6.203629
...,...,...
3223,46.384180,6.245370
3224,46.385209,6.921363
3225,46.035701,8.919653
3226,46.215171,6.104451


### Plot the charging stations of the operator 'evepass'

In [16]:
# Subset of gas stations by brand
locations = df[["lat", "lon"]]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]]).add_to(map)

# Plot map
map

         lat       lon
0  46.547745  6.552618
1  46.547745  6.552618
2  46.308936  7.471237
3  46.308936  7.471237
4  46.222434  6.203629


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [17]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2024-04-17 12:18:26
Python Version: 3.11.7
-----------------------------------
